In [1]:
import numpy as np
import pandas as pd 
import os
from pandas.tseries.holiday import USFederalHolidayCalendar
from glob import glob
import matplotlib.pyplot as plt

import utils
# from utils import load_data, get_train_val_split, get_stratified_splitter
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedGroupKFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from lightgbm import LGBMRegressor
from scipy.stats import kstest, kruskal, mannwhitneyu
from itertools import combinations
from collections import defaultdict
from tqdm import tqdm
from sklearn.ensemble import StackingRegressor
import re
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_log_error, mean_squared_error, silhouette_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

import optuna
import pickle

In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from '/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py'>

In [3]:
data_dict = utils.load_data('ashrae-energy-prediction')

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.88%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 3.07 MB
Decreased by 68.05%
Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 5.13 MB
Decreased by 73.04%
Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.12%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.53 MB
Decreased by 71.82%


In [4]:
# Add weather features 
weather_features = ['cloud_coverage', 'dew_temperature', 'air_temperature', 
                    'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr',]

hourly_by_site = data_dict["X_train"].groupby(['hour', 'month', 'site_id'])[weather_features].mean().reset_index()

data_dict["X_train"] = data_dict["X_train"].merge(
    hourly_by_site, 
    on=['hour', 'month', 'site_id'], 
    how='left', 
    suffixes=(None, '_hourly_by_site')
)

del hourly_by_site

for feature in weather_features:
    # Fill in NA values from weather with hourly by site columns 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature + "_hourly_by_site"],
        inplace=True
    )
    
    # Fill in the rest with the median 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(),
        inplace=True
    )
    
    data_dict["X_train"][feature + "_diff_hourly_from_mean"] = data_dict["X_train"][feature] - \
        data_dict["X_train"][feature + "_hourly_by_site"]
    
data_dict["X_train"] = data_dict["X_train"].drop(columns = [feat + "_hourly_by_site" for feat in weather_features])

In [5]:
# Fill in NA with median values for floor count and year_built
for feature in ['year_built', 'floor_count']:
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(), 
        inplace=True
    )

In [6]:
data_dict.keys()

dict_keys(['weather_test', 'X_train', 'X_test', 'y_train'])

## Examine Differences (Non-Parametric)
Using Bonferonni's Correction

### Milestone 2. 
- Show difference in sites across meter readings 
- get average meter reading per day per site 
- conduct a (non-parametric ANOVA) KS OR pairs (mann-whitney) to show that they are diff 
- train a model per site id (with rudimentary hyperparameter tuning) 
- John sites 0-7, Sharad sites 8-15 

### Milestone 3. Determine, per site, which primary uses are similar (if they have only a few buildings), which are diff
- for a given primary use, if diff, identify "clusters" of buildings that are similar 

In [7]:
features = ['year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed',
       'air_temperature_mean_lag7', 'air_temperature_max_lag7',
       'air_temperature_min_lag7', 'air_temperature_std_lag7',
       'cloud_coverage_mean_lag7', 'cloud_coverage_max_lag7',
       'cloud_coverage_min_lag7', 'cloud_coverage_std_lag7',
       'dew_temperature_mean_lag7', 'dew_temperature_max_lag7',
       'dew_temperature_min_lag7', 'dew_temperature_std_lag7',
       'precip_depth_1_hr_mean_lag7', 'precip_depth_1_hr_max_lag7',
       'precip_depth_1_hr_min_lag7', 'precip_depth_1_hr_std_lag7',
       'sea_level_pressure_mean_lag7', 'sea_level_pressure_max_lag7',
       'sea_level_pressure_min_lag7', 'sea_level_pressure_std_lag7',
       'wind_direction_mean_lag7', 'wind_direction_max_lag7',
       'wind_direction_min_lag7', 'wind_direction_std_lag7',
       'wind_speed_mean_lag7', 'wind_speed_max_lag7', 'wind_speed_min_lag7',
       'wind_speed_std_lag7', 'log_square_feet', 'weekday', 'hour', 'day',
       'weekend', 'month', 'primary_use_enc']

In [8]:
X, y = data_dict['X_train'].loc[:, features+['meter', 'site_id', 'building_id', 'timestamp']], data_dict['y_train']
splitter_gen = utils.get_stratified_splitter(X, y)
train_index, test_index = next(splitter_gen)
X_train, y_train, X_test, y_test = X.loc[train_index, :], y[train_index], X.loc[test_index, :], y[test_index]

In [9]:
building_cluster_mapping = pd.DataFrame()

silhouette_scores = {}
for site in X_train['site_id'].unique():
    for meter in X_train['meter'].unique():
        X = X_train[(X_train["site_id"]==site)&(X_train["meter"]==meter)]
        y = y_train[(X_train["site_id"]==site)&(X_train["meter"]==meter)]
        X["meter_reading"] = y

        X["YMD"] = X["timestamp"].dt.strftime("%Y-%m-%d")
        day_level_df = X.groupby(["building_id", "YMD"]).agg(meter_reading = ("meter_reading", "mean")).reset_index()
        day_level_pivot = pd.pivot_table(day_level_df, values="meter_reading", columns="YMD", index="building_id").fillna(0)
        if X.shape[0]==0:
            continue
        
        
        key = "_".join([str(site), str(meter)])
        
        silhouette_scores[key] = []
        n_buildings = day_level_pivot.shape[0]
        n_pc_range = range(2, min(n_buildings - 1, 10))
        k_range = range(2, min(10, n_buildings - 1))
        #Default best_k, best_pc values
        best_k, best_pc, best_score, best_labels = 1, 2, 0, np.ones(n_buildings)
        for n_pc in n_pc_range:
            pca = PCA(n_pc)
            t = pca.fit_transform(day_level_pivot)
            for k in k_range:
                kmeans_labels = KMeans(n_clusters=k, random_state=0).fit(t[:, :n_pc]).labels_
                ss = silhouette_score(t[:, :n_pc], kmeans_labels)
                if ss > best_score:
                    best_pc, best_k, best_score = n_pc, k, ss
                    best_labels = kmeans_labels
        
        print(site, meter, best_pc, best_k)
        
        day_level_pivot["cluster"] = best_labels
        day_level_pivot = day_level_pivot.reset_index()
        bc_mapping = day_level_pivot[["building_id", "cluster"]]
        bc_mapping["meter"] = meter
        building_cluster_mapping = pd.concat([building_cluster_mapping, bc_mapping])

1 0 3 4
1 3 2 2
2 0 2 2
2 3 2 2
2 1 2 2
3 0 2 4
4 0 2 2
5 0 2 2
6 0 2 3
6 1 2 6
6 2 2 3
7 0 2 2
7 3 2 1
7 1 2 2
7 2 2 3
8 0 2 9
9 0 2 2
9 1 2 3
9 2 2 4
10 0 2 2
10 3 2 2
10 1 2 2
11 0 2 1
11 3 2 2
11 1 2 1
12 0 2 7
13 0 2 6
13 1 2 2
13 2 2 2
14 0 2 8
14 3 2 2
14 1 2 2
14 2 2 4
15 0 2 2
15 3 2 1
15 1 2 2
15 2 2 2
0 0 2 6
0 1 2 3


In [10]:
X_train[X_train["building_id"]==1442]["site_id"].unique()

array([15], dtype=int8)

In [11]:
X_train_w_cluster = X_train.reset_index().merge(building_cluster_mapping, on=["building_id", "meter"], how="left").set_index('index')

In [12]:
X_train_w_cluster.shape

(14892330, 49)

In [13]:
SITE_FILTER = X_train_w_cluster["site_id"] == 1
METER_FILTER = X_train_w_cluster["meter"] == 1

SITE_FILTER&METER_FILTER

index
0           False
2           False
3           False
7           False
8           False
            ...  
19852312    False
19852313    False
19852314    False
19852315    False
19852319    False
Length: 14892330, dtype: bool

In [41]:
def run_optuna_search_cv(
    site: int,
    meter: int,
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    features: list,
    n_trials: int = 50,
    
):
    """
    Runs Optuna Search for LGBMRegressor
    """
    SITE_FILTER = X_train["site_id"] == site
    METER_FILTER = X_train["meter"] == meter
    
    X, y = X_train.loc[SITE_FILTER&METER_FILTER, features], y_train[SITE_FILTER&METER_FILTER]
    if X.shape[0] == 0:
        return None

    splitter_gen = utils.get_stratified_splitter(X_train[SITE_FILTER&METER_FILTER], y_train[SITE_FILTER&METER_FILTER])

    

    regressor = LGBMRegressor()

    param_distributions = {
        "max_depth": optuna.distributions.IntDistribution(-1, len(X_train.columns)),
        "num_leaves": optuna.distributions.IntDistribution(5, 50),
        "learning_rate": optuna.distributions.FloatDistribution(1e-7, 1, log=True),
        "n_estimators": optuna.distributions.IntDistribution(1, 500),
        "reg_alpha": optuna.distributions.FloatDistribution(1e-7, 1e7, log=True),
        "reg_lambda": optuna.distributions.FloatDistribution(1e-7, 1e7, log=True),            
    }

    """
    Parameters not searched over: 
    subsample_for_bin: int = 200000,
    min_split_gain: float = 0.0,
    min_child_weight: float = 0.001,
    min_child_samples: int = 20,
    subsample: float = 1.0,
    subsample_freq: int = 0,
    colsample_bytree: float = 1.0,
    random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None,
    n_jobs: int = -1,
    """
    def rmse(estimator, X_test, y_test):
        y_pred = estimator.predict(X_test)
        return -1 * mean_squared_error(y_test, y_pred, squared=False)
    
    
    optuna_search = optuna.integration.OptunaSearchCV(
        regressor, 
        param_distributions,
        n_trials=n_trials,
        cv = splitter_gen,
        random_state=0, # IMPORTANT,
        refit=True,
        n_jobs=4,
        scoring = rmse,
        verbose=optuna.logging.ERROR
    )

    optuna_search.fit(X, y)
    y_pred = optuna_search.predict(X)
    
    return optuna_search

In [42]:
# This fits sites 0-7
N_TRIALS = 50
models = defaultdict(dict)

for site in tqdm(X_train["site_id"].unique()):
    for meter in X_train["meter"].unique():
        clusters = X_train_w_cluster.loc[(X_train_w_cluster["site_id"]==site)&(X_train_w_cluster["meter"]==meter), "cluster"].unique()
        for cluster in clusters:
            optuna_search = run_optuna_search_cv(site, \
                                                 meter, \
                                                 X_train_w_cluster[X_train_w_cluster["cluster"] == cluster], \
                                                 y_train[X_train_w_cluster["cluster"] == cluster], \
                                                 features, \
                                                 N_TRIALS)

            model_identifier = "_".join([str(site), str(meter), str(cluster)])
            models[model_identifier] = optuna_search

  0%|                                                                             | 0/16 [00:00<?, ?it/s][I 2023-04-28 01:01:23,272] A new study created in memory with name: no-name-cfdf0f0b-88a2-4b85-b24d-5e4e6bb0f088
[I 2023-04-28 01:01:23,275] Searching the best hyperparameters using 166896 samples...
[I 2023-04-28 01:01:24,738] Trial 0 finished with value: -0.5940301634661521 and parameters: {'max_depth': 46, 'num_leaves': 18, 'learning_rate': 1.1582164435988091e-06, 'n_estimators': 32, 'reg_alpha': 40944.00461498572, 'reg_lambda': 0.00756810171752901}. Best is trial 0 with value: -0.5940301634661521.
[I 2023-04-28 01:01:25,823] Trial 3 finished with value: -0.5482540117402765 and parameters: {'max_depth': 3, 'num_leaves': 35, 'learning_rate': 0.04976845793864109, 'n_estimators': 53, 'reg_alpha': 0.0006727035303439568, 'reg_lambda': 11.904660564497407}. Best is trial 3 with value: -0.5482540117402765.
[I 2023-04-28 01:01:26,976] Trial 2 finished with value: -0.5940301634661521 and 

[I 2023-04-28 01:03:42,525] Trial 23 finished with value: -0.6982440606762028 and parameters: {'max_depth': 8, 'num_leaves': 41, 'learning_rate': 0.8859730698668187, 'n_estimators': 389, 'reg_alpha': 4.049016976476346e-06, 'reg_lambda': 209.50768273600275}. Best is trial 15 with value: -0.5275789337922094.
[I 2023-04-28 01:03:47,884] Trial 27 finished with value: -0.6297408851952788 and parameters: {'max_depth': 0, 'num_leaves': 29, 'learning_rate': 0.08624379505177743, 'n_estimators': 175, 'reg_alpha': 3.567176626548637e-06, 'reg_lambda': 221.38093113406475}. Best is trial 15 with value: -0.5275789337922094.
[I 2023-04-28 01:03:55,294] Trial 24 finished with value: -0.6676249008341786 and parameters: {'max_depth': -1, 'num_leaves': 40, 'learning_rate': 0.0473092036421708, 'n_estimators': 378, 'reg_alpha': 1.2038559006510112e-05, 'reg_lambda': 73.08513345112884}. Best is trial 15 with value: -0.5275789337922094.
[I 2023-04-28 01:04:04,510] Trial 28 finished with value: -0.6954323796024

[I 2023-04-28 01:05:50,156] Finished refitting! (elapsed time: 5.375 sec.)
[I 2023-04-28 01:05:52,136] A new study created in memory with name: no-name-823e496f-1db3-4d7c-a573-16fd1a5ae8e9
[I 2023-04-28 01:05:52,138] Searching the best hyperparameters using 8784 samples...
[W 2023-04-28 01:05:52,291] Trial 1 failed with parameters: {'max_depth': 15, 'num_leaves': 7, 'learning_rate': 0.0012916916897495045, 'n_estimators': 11, 'reg_alpha': 1578051.3704075438, 'reg_lambda': 0.0005463701061119398} because of the following error: The value nan is not acceptable..
[W 2023-04-28 01:05:52,292] Trial 1 failed with value nan.
[W 2023-04-28 01:05:53,582] Trial 4 failed with parameters: {'max_depth': 46, 'num_leaves': 26, 'learning_rate': 0.01334440183855473, 'n_estimators': 168, 'reg_alpha': 26.83680779903621, 'reg_lambda': 816917.8707546919} because of the following error: The value nan is not acceptable..
[W 2023-04-28 01:05:53,583] Trial 4 failed with value nan.
[W 2023-04-28 01:05:54,193] Tri

[W 2023-04-28 01:06:16,670] Trial 19 failed with parameters: {'max_depth': 11, 'num_leaves': 39, 'learning_rate': 0.026789273574340855, 'n_estimators': 223, 'reg_alpha': 1.3933091897633561e-05, 'reg_lambda': 0.0290586975349052} because of the following error: The value nan is not acceptable..
[W 2023-04-28 01:06:16,671] Trial 19 failed with value nan.
[W 2023-04-28 01:06:16,829] Trial 25 failed with parameters: {'max_depth': 25, 'num_leaves': 28, 'learning_rate': 0.0688467642452289, 'n_estimators': 48, 'reg_alpha': 6239406.938643455, 'reg_lambda': 20.192361431529996} because of the following error: The value nan is not acceptable..
[W 2023-04-28 01:06:16,830] Trial 25 failed with value nan.
[W 2023-04-28 01:06:17,584] Trial 26 failed with parameters: {'max_depth': 20, 'num_leaves': 15, 'learning_rate': 1.9275231864423944e-07, 'n_estimators': 277, 'reg_alpha': 44889.355658725915, 'reg_lambda': 6.1231915872027844} because of the following error: The value nan is not acceptable..
[W 2023-

[W 2023-04-28 01:06:36,246] Trial 39 failed with value nan.
[W 2023-04-28 01:06:36,454] Trial 46 failed with parameters: {'max_depth': 11, 'num_leaves': 47, 'learning_rate': 0.062122068190404854, 'n_estimators': 58, 'reg_alpha': 6062712.81281612, 'reg_lambda': 0.008579252535118749} because of the following error: The value nan is not acceptable..
[W 2023-04-28 01:06:36,455] Trial 46 failed with value nan.
[W 2023-04-28 01:06:36,648] Trial 47 failed with parameters: {'max_depth': 31, 'num_leaves': 6, 'learning_rate': 0.009797892092399223, 'n_estimators': 16, 'reg_alpha': 0.017213538356121317, 'reg_lambda': 10.70528886280495} because of the following error: The value nan is not acceptable..
[W 2023-04-28 01:06:36,649] Trial 47 failed with value nan.
[W 2023-04-28 01:06:36,882] Trial 48 failed with parameters: {'max_depth': 38, 'num_leaves': 26, 'learning_rate': 2.8722269677717754e-07, 'n_estimators': 73, 'reg_alpha': 14202.470832701498, 'reg_lambda': 0.00038654426931014873} because of th

...

[I 2023-04-28 07:12:17,988] A new study created in memory with name: no-name-3f562717-e313-4533-81c2-0f5022c408b4
[I 2023-04-28 07:12:17,989] Searching the best hyperparameters using 40542 samples...
[I 2023-04-28 07:12:19,719] Trial 2 finished with value: -2.9332533667297755 and parameters: {'max_depth': 16, 'num_leaves': 41, 'learning_rate': 3.1419524630588186e-06, 'n_estimators': 342, 'reg_alpha': 2970243.8000275213, 'reg_lambda': 1.8055525089145556e-07}. Best is trial 2 with value: -2.9332533667297755.
[I 2023-04-28 07:12:19,801] Trial 0 finished with value: -2.9332125625114767 and parameters: {'max_depth': 24, 'num_leaves': 8, 'learning_rate': 2.8662500975488775e-06, 'n_estimators': 79, 'reg_alpha': 1.7367942956714757, 'reg_lambda': 1.2310603566656433e-07}. Best is trial 0 with value: -2.9332125625114767.
[I 2023-04-28 07:12:20,282] Trial 4 finished with value: -2.9332533667297755 and parameters: {'max_depth': 24, 'num_leaves': 40, 'learning_rate': 7.456692658439094e-05, 'n_estima

[I 2023-04-28 07:14:02,763] Trial 25 finished with value: -2.90328280026775 and parameters: {'max_depth': 49, 'num_leaves': 20, 'learning_rate': 0.0004511224357128696, 'n_estimators': 389, 'reg_alpha': 3.1821396685150866e-05, 'reg_lambda': 0.0013021838915741211}. Best is trial 25 with value: -2.90328280026775.
[I 2023-04-28 07:14:15,514] Trial 26 finished with value: -2.9095091178070067 and parameters: {'max_depth': 43, 'num_leaves': 23, 'learning_rate': 0.0006013480430047754, 'n_estimators': 305, 'reg_alpha': 9.809435770926697e-06, 'reg_lambda': 0.0013266815423268888}. Best is trial 25 with value: -2.90328280026775.
[I 2023-04-28 07:14:23,981] Trial 27 finished with value: -2.910089851981174 and parameters: {'max_depth': 39, 'num_leaves': 34, 'learning_rate': 0.0003148391002263222, 'n_estimators': 290, 'reg_alpha': 3.8242499476818674e-05, 'reg_lambda': 0.002554617091622913}. Best is trial 25 with value: -2.90328280026775.
[I 2023-04-28 07:14:32,394] Trial 29 finished with value: -2.90

[I 2023-04-28 07:16:34,316] Finished refitting! (elapsed time: 1.920 sec.)
[I 2023-04-28 07:16:35,612] A new study created in memory with name: no-name-d5bf5b1b-3220-4848-8a6c-36f5076217e1
[I 2023-04-28 07:16:35,615] Searching the best hyperparameters using 5147 samples...
[W 2023-04-28 07:16:35,720] Trial 2 failed with parameters: {'max_depth': 45, 'num_leaves': 14, 'learning_rate': 1.0141008392884922e-06, 'n_estimators': 10, 'reg_alpha': 377.0702197618795, 'reg_lambda': 0.0077654227457758015} because of the following error: The value nan is not acceptable..
[W 2023-04-28 07:16:35,721] Trial 2 failed with value nan.
[W 2023-04-28 07:16:36,503] Trial 0 failed with parameters: {'max_depth': 45, 'num_leaves': 14, 'learning_rate': 0.003712006241873784, 'n_estimators': 119, 'reg_alpha': 117.94935908084284, 'reg_lambda': 39295.258537210146} because of the following error: The value nan is not acceptable..
[W 2023-04-28 07:16:36,504] Trial 0 failed with value nan.
[W 2023-04-28 07:16:37,868]

[W 2023-04-28 07:17:09,208] Trial 21 failed with parameters: {'max_depth': 30, 'num_leaves': 31, 'learning_rate': 0.00766988632281103, 'n_estimators': 299, 'reg_alpha': 0.004000152973429481, 'reg_lambda': 2.8958064360292395e-06} because of the following error: The value nan is not acceptable..
[W 2023-04-28 07:17:09,209] Trial 21 failed with value nan.
[W 2023-04-28 07:17:09,308] Trial 24 failed with parameters: {'max_depth': 23, 'num_leaves': 12, 'learning_rate': 0.0008820355512307212, 'n_estimators': 10, 'reg_alpha': 0.5984891513678969, 'reg_lambda': 238956.0597040836} because of the following error: The value nan is not acceptable..
[W 2023-04-28 07:17:09,309] Trial 24 failed with value nan.
[W 2023-04-28 07:17:11,272] Trial 18 failed with parameters: {'max_depth': 33, 'num_leaves': 29, 'learning_rate': 0.0005690345850033038, 'n_estimators': 411, 'reg_alpha': 2.695531950860208e-06, 'reg_lambda': 2201.7788572543886} because of the following error: The value nan is not acceptable..
[W

[W 2023-04-28 07:17:40,376] Trial 45 failed with value nan.
[W 2023-04-28 07:17:44,669] Trial 43 failed with parameters: {'max_depth': 15, 'num_leaves': 14, 'learning_rate': 6.104527400870275e-05, 'n_estimators': 376, 'reg_alpha': 0.000747239090652883, 'reg_lambda': 0.005451228388924159} because of the following error: The value nan is not acceptable..
[W 2023-04-28 07:17:44,669] Trial 43 failed with value nan.
[W 2023-04-28 07:17:45,656] Trial 47 failed with parameters: {'max_depth': 45, 'num_leaves': 6, 'learning_rate': 0.3939445070228184, 'n_estimators': 95, 'reg_alpha': 8.000055419646751e-07, 'reg_lambda': 0.01039518541992548} because of the following error: The value nan is not acceptable..
[W 2023-04-28 07:17:45,657] Trial 47 failed with value nan.
[W 2023-04-28 07:17:46,215] Trial 36 failed with parameters: {'max_depth': 18, 'num_leaves': 49, 'learning_rate': 4.165062797824824e-07, 'n_estimators': 281, 'reg_alpha': 0.03688239244317533, 'reg_lambda': 2.1633964022528245} because o

###### Dump the models

In [43]:
for key in models.keys():
    if models[key]:
        with open(f"./models/v4/{key}.pkl", "wb") as f:
            pickle.dump(models[key].best_estimator_, f)

In [47]:
len(models.keys())

117

In [55]:
model_files = sorted(glob('models/v4/*.pkl'))
estimators = []
for model_file in model_files:
    estimators.append((re.search(r"/(\d+_\d+_\d+)\.0\.pkl$", model_file).group(1) ,pickle.load(open(model_file,'rb'))))

In [57]:
estimators_dict = dict(estimators)

In [64]:
def predict_using_individual_estimators(X_test, y_test, estimators_dict):
    results_df = pd.DataFrame(columns=['y_true', 'y_pred'])
    for site_meter_cluster in estimators_dict.keys():

        site_id, meter, cluster = list(map(lambda x:int(x) ,site_meter_cluster.split("_")))
        # Filter input data for the specific site_id and meter
        site_meter_filter = (X_test['site_id'] == site_id) & (X_test['meter'] == meter) & (X_test['cluster'] == cluster)

        X_test_subset = X_test[site_meter_filter].drop(['meter', 'site_id', 'building_id', 'timestamp'], axis=1)

        # Get the true values for the subset of input data
        y_true = y_test[site_meter_filter]
        
        print(X_test_subset.shape)
        print(y_true.shape)
        if(X_test_subset.shape[0] == 0):
            continue

        # Make predictions for the subset of input data
        y_pred = estimators_dict[site_meter].predict(X_test_subset)

        # Add true values and predictions to results_df
        results_df = pd.concat([results_df, pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})], ignore_index=True)

        # Compute RMSE for the predictions and print the results
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        print(f"Site ID: {site_id}, Meter: {meter}, RMSE: {rmse}")
    overall_rmse = mean_squared_error(results_df["y_true"], results_df["y_pred"], squared=False)
    print(f"Overall RMSE: {overall_rmse}")
    return results_df

In [65]:
X_test_w_cluster = X_test.reset_index().merge(building_cluster_mapping, on=["building_id", "meter"], how="left").set_index('index')


In [73]:
site_id, meter, cluster = list(map(lambda x:int(x), list(estimators_dict.keys())[0].split("_")))

In [ ]:
results_df = predict_using_individual_estimators(X_test_w_cluster, y_test, estimators_dict)

In [34]:
results_df = predict_using_individual_estimators(X_train_w_cluster, y_train, estimators_dict)

Site ID: 0, Meter: 0, RMSE: 0.29867332586344236
Site ID: 0, Meter: 1, RMSE: 2.157743845815844
Site ID: 10, Meter: 0, RMSE: 0.815686384558406
Site ID: 10, Meter: 1, RMSE: 1.0831839961711274
Site ID: 10, Meter: 3, RMSE: 1.8436743118176624
Site ID: 11, Meter: 0, RMSE: 0.048857680003833164
Site ID: 11, Meter: 1, RMSE: 0.9206727188179956
Site ID: 11, Meter: 3, RMSE: 1.04811202602139
Site ID: 12, Meter: 0, RMSE: 0.4468469928198841
Site ID: 13, Meter: 0, RMSE: 0.5183147105153256
Site ID: 13, Meter: 1, RMSE: 1.320765307926928
Site ID: 13, Meter: 2, RMSE: 1.6221471577724524
Site ID: 14, Meter: 0, RMSE: 0.6618018007034662
Site ID: 14, Meter: 1, RMSE: 1.7158792737887012
Site ID: 14, Meter: 2, RMSE: 1.058148859418716
Site ID: 14, Meter: 3, RMSE: 1.965487228670819
Site ID: 15, Meter: 0, RMSE: 0.479894143058484
Site ID: 15, Meter: 1, RMSE: 0.9424270055126931
Site ID: 15, Meter: 2, RMSE: 1.0050557036402312
Site ID: 15, Meter: 3, RMSE: 1.3803327979528004
Site ID: 1, Meter: 0, RMSE: 0.6247277373585172


In [84]:
best_params_df = pd.DataFrame()
for key in models.keys():
    if models[key]:
        try:
            params = models[key].best_params_
            params["best_score"] = models[key].best_score_
        except:
            params = {}
        params["site_id"] = key.split("_")[0]
        params["meter"] = key.split("_")[1]
        best_params_df = pd.concat([best_params_df, pd.DataFrame(params, index=[key])])

In [85]:
best_params_df.to_csv("BestParamsWithClusterLevel.csv")

In [61]:
models["11_0"].trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_n_estimators,params_num_leaves,params_reg_alpha,...,user_attrs_mean_score_time,user_attrs_mean_test_score,user_attrs_split0_test_score,user_attrs_split1_test_score,user_attrs_split2_test_score,user_attrs_split3_test_score,user_attrs_std_fit_time,user_attrs_std_score_time,user_attrs_std_test_score,state
0,0,None,2023-04-25 19:15:52.843865,2023-04-25 19:15:57.533979,0 days 00:00:04.690114,2.853813e-06,43,385,18,9.046719e+02,...,0.000227,NaN,NaN,NaN,NaN,NaN,0.681406,0.000168,NaN,FAIL
1,1,None,2023-04-25 19:15:52.850513,2023-04-25 19:16:04.086423,0 days 00:00:11.235910,6.392680e-03,39,298,24,1.626622e-01,...,0.000510,NaN,NaN,NaN,NaN,NaN,1.613155,0.000602,NaN,FAIL
2,2,None,2023-04-25 19:15:52.852508,2023-04-25 19:15:54.727451,0 days 00:00:01.874943,7.453219e-03,34,415,46,2.997876e+05,...,0.000710,NaN,NaN,NaN,NaN,NaN,0.278675,0.000984,NaN,FAIL
3,3,None,2023-04-25 19:15:52.854734,2023-04-25 19:16:20.864823,0 days 00:00:28.010089,3.160354e-03,14,443,35,3.560444e+00,...,0.000177,NaN,NaN,NaN,NaN,NaN,4.073371,0.000104,NaN,FAIL
4,4,None,2023-04-25 19:15:54.730126,2023-04-25 19:15:55.835744,0 days 00:00:01.105618,1.172477e-02,3,70,25,2.412308e-05,...,0.000223,NaN,NaN,NaN,NaN,NaN,0.168333,0.000162,NaN,FAIL
5,5,None,2023-04-25 19:15:55.838492,2023-04-25 19:15:59.059942,0 days 00:00:03.221450,8.678914e-04,42,268,50,1.215929e+03,...,0.000179,NaN,NaN,NaN,NaN,NaN,0.485249,0.000106,NaN,FAIL
6,6,None,2023-04-25 19:15:57.536587,2023-04-25 19:16:13.816861,0 days 00:00:16.280274,7.936785e-04,31,239,37,2.898895e-06,...,0.000162,NaN,NaN,NaN,NaN,NaN,2.354601,0.000094,NaN,FAIL
7,7,None,2023-04-25 19:15:59.062499,2023-04-25 19:16:07.901796,0 days 00:00:08.839297,1.353845e-02,30,298,16,1.704116e-06,...,0.000538,NaN,NaN,NaN,NaN,NaN,1.276135,0.000653,NaN,FAIL
8,8,None,2023-04-25 19:16:04.088592,2023-04-25 19:16:04.250656,0 days 00:00:00.162064,2.408493e-04,31,40,46,8.211615e+04,...,0.000193,NaN,NaN,NaN,NaN,NaN,0.022451,0.000114,NaN,FAIL
9,9,None,2023-04-25 19:16:04.252953,2023-04-25 19:16:17.565311,0 days 00:00:13.312358,1.024572e-04,39,459,19,1.019510e+01,...,0.000165,NaN,NaN,NaN,NaN,NaN,1.922425,0.000095,NaN,FAIL


In [86]:
trials_df = pd.DataFrame()
for key in models.keys():
    if models[key]:
        trial_df = models[key].trials_dataframe()
        trial_df["key"] = key
        trials_df = pd.concat([trials_df, trial_df])

In [87]:
trials_df.to_csv("OptunaStudyTrialsAllWithClusterLevelModels.csv", index=False)